<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [47]:
import skimage
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
from skimage import io

In [15]:
forests = skimage.io.imread_collection("data//forest//*.jpg")

In [17]:
forests.files[:10]

['data//forest\\art114.jpg',
 'data//forest\\bost98.jpg',
 'data//forest\\bost100.jpg',
 'data//forest\\bost101.jpg',
 'data//forest\\bost102.jpg',
 'data//forest\\bost103.jpg',
 'data//forest\\bost190.jpg',
 'data//forest\\cdmc12.jpg',
 'data//forest\\cdmc101.jpg',
 'data//forest\\cdmc271.jpg']

In [18]:
forests=forests.concatenate()
forests.shape

(328, 256, 256, 3)

In [19]:
mountains = skimage.io.imread_collection("data//mountain//*.jpg")
mountains = mountains.concatenate()
mountains.shape

(374, 256, 256, 3)

In [29]:
X = np.concatenate((forests, mountains))
X.shape

(702, 256, 256, 3)

In [30]:
y = np.array([0] * 328 + [1] * 374)
y.shape

(702,)

In [31]:
X[1]

array([[[ 85,  89,  62],
        [ 96, 100,  75],
        [ 83,  82,  61],
        ...,
        [ 62,  65,  48],
        [ 29,  33,  18],
        [ 52,  56,  41]],

       [[ 97, 101,  74],
        [128, 132, 107],
        [119, 118,  97],
        ...,
        [118, 121, 104],
        [ 87,  91,  76],
        [ 72,  76,  61]],

       [[ 97, 101,  74],
        [141, 145, 120],
        [121, 120,  99],
        ...,
        [ 76,  80,  63],
        [ 88,  92,  77],
        [ 91,  95,  80]],

       ...,

       [[120, 125,  58],
        [163, 168, 101],
        [174, 179, 112],
        ...,
        [162, 166, 107],
        [171, 175, 116],
        [126, 130,  71]],

       [[162, 167, 101],
        [188, 193, 127],
        [194, 201, 134],
        ...,
        [138, 142,  83],
        [161, 164, 107],
        [131, 134,  77]],

       [[130, 135,  71],
        [130, 135,  71],
        [124, 130,  66],
        ...,
        [116, 120,  61],
        [138, 141,  84],
        [102, 105,  48]]

For optimal processing, we scale the values down by 255 to set them between 0 and 1.

In [37]:
X_t = X/255

We instantialize a pre-trained instance of the ResNet model. We remove the fully-connected layers so that we can train those layers on our own data set.

In [5]:
resnet = ResNet50(weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 5s 0us/step


I'm setting the layers to non-trainable -- for computational purposes, we want to use the learned parameters without updating them in future passes.

In [6]:
for layer in resnet.layers:
    layer.trainable = False


In [ ]:
We then 

In [33]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
#(fancy flattening)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [35]:
model.compile(optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
model.fit(X, y, epochs=20, validation_split=.2)

Train on 561 samples, validate on 141 samples
Epoch 1/20
561/561 [==============================] - 15s 27ms/sample - loss: 0.5232 - accuracy: 0.8984 - val_loss: 7.0171 - val_accuracy: 0.0000e+00
Epoch 2/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.1171 - accuracy: 0.9697 - val_loss: 0.9087 - val_accuracy: 0.6525
Epoch 3/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.0222 - accuracy: 0.9947 - val_loss: 0.7800 - val_accuracy: 0.7021
Epoch 4/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.0183 - accuracy: 0.9947 - val_loss: 0.2809 - val_accuracy: 0.8794
Epoch 5/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.0164 - accuracy: 0.9947 - val_loss: 1.2282 - val_accuracy: 0.6596
Epoch 6/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.0144 - accuracy: 0.9964 - val_loss: 0.6511 - val_accuracy: 0.7872
Epoch 7/20
561/561 [==============================] - 5s 9ms/sample - loss: 0.0036 -

The final model yields a validation accuracy of 87.2% (and is perfect on the test data set). The best model had a validation accuracy of 96%.

In [39]:
model.evaluate(X, y, batch_size=10, steps=10)

100/702 [===>..........................] - ETA: 8s - loss: 2.6345e-05 - accuracy: 1.0000

[3.7528810278402177e-06, 1.0]

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [49]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)              

In [50]:
# Compile Model
model.compile(optimizer='nadam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [52]:
stop = EarlyStopping(monitor='val_accuracy', min_delta=.01, patience=6)
model.fit(X, y, epochs=25, validation_split=.2, callbacks=[stop])

Train on 561 samples, validate on 141 samples
Epoch 1/25
561/561 [==============================] - 3s 5ms/sample - loss: 0.0231 - accuracy: 0.9947 - val_loss: 4.1981 - val_accuracy: 0.5461
Epoch 2/25
561/561 [==============================] - 3s 5ms/sample - loss: 0.0059 - accuracy: 0.9982 - val_loss: 4.3557 - val_accuracy: 0.5887
Epoch 3/25
561/561 [==============================] - 2s 4ms/sample - loss: 0.0311 - accuracy: 0.9964 - val_loss: 5.8972 - val_accuracy: 0.4823
Epoch 4/25
561/561 [==============================] - 3s 5ms/sample - loss: 0.0124 - accuracy: 0.9982 - val_loss: 3.2247 - val_accuracy: 0.6099
Epoch 5/25
561/561 [==============================] - 3s 5ms/sample - loss: 0.0041 - accuracy: 1.0000 - val_loss: 3.5944 - val_accuracy: 0.6099
Epoch 6/25
561/561 [==============================] - 3s 4ms/sample - loss: 0.0050 - accuracy: 0.9982 - val_loss: 3.4116 - val_accuracy: 0.6028
Epoch 7/25
561/561 [==============================] - 3s 4ms/sample - loss: 0.0018 - accur

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [54]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [55]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [56]:
datagen.fit(X)

In [60]:
model.fit(datagen.flow(X, y, batch_size=32), steps_per_epoch=20, epochs=25)

  ...
    to  
  ['...']
Train for 20 steps
Epoch 1/25
20/20 [==============================] - 9s 474ms/step - loss: 0.1440 - accuracy: 0.9436
Epoch 2/25
20/20 [==============================] - 9s 459ms/step - loss: 0.1273 - accuracy: 0.9530
Epoch 3/25
20/20 [==============================] - 9s 462ms/step - loss: 0.1544 - accuracy: 0.9373
Epoch 4/25
20/20 [==============================] - 13s 644ms/step - loss: 0.1136 - accuracy: 0.9577
Epoch 5/25
20/20 [==============================] - 9s 467ms/step - loss: 0.1290 - accuracy: 0.9469
Epoch 6/25
20/20 [==============================] - 10s 481ms/step - loss: 0.1467 - accuracy: 0.9420
Epoch 7/25
20/20 [==============================] - 9s 475ms/step - loss: 0.1438 - accuracy: 0.9641
Epoch 8/25
20/20 [==============================] - 9s 457ms/step - loss: 0.1226 - accuracy: 0.9453
Epoch 9/25
20/20 [==============================] - 9s 462ms/step - loss: 0.1417 - accuracy: 0.9545
Epoch 10/25
20/20 [==============================] - 9

2020-03-24 17:01:58.094134: W tensorflow/core/kernels/data/generator_dataset_op.cc:103] Error occurred when finalizing GeneratorDataset iterator: Cancelled: Operation was cancelled

Per documentation, make the below change to avoid error 

In [61]:
model.fit(datagen.flow(X, y, batch_size=32), steps_per_epoch=len(X)/32, epochs=25)

  ...
    to  
  ['...']
Train for 21.9375 steps
Epoch 1/25
22/21 [==============================] - 12s 527ms/step - loss: 0.0993 - accuracy: 0.9573
Epoch 2/25
22/21 [==============================] - 11s 500ms/step - loss: 0.1195 - accuracy: 0.9501
Epoch 3/25
22/21 [==============================] - 11s 497ms/step - loss: 0.1306 - accuracy: 0.9501
Epoch 4/25
22/21 [==============================] - 12s 545ms/step - loss: 0.1249 - accuracy: 0.9516
Epoch 5/25
22/21 [==============================] - 13s 578ms/step - loss: 0.1348 - accuracy: 0.9473
Epoch 6/25
22/21 [==============================] - 11s 519ms/step - loss: 0.1234 - accuracy: 0.9573
Epoch 7/25
22/21 [==============================] - 12s 527ms/step - loss: 0.1289 - accuracy: 0.9615
Epoch 8/25
22/21 [==============================] - 12s 526ms/step - loss: 0.1086 - accuracy: 0.9544
Epoch 9/25
22/21 [==============================] - 11s 516ms/step - loss: 0.0963 - accuracy: 0.9644
Epoch 10/25
22/21 [=======================

How to tell if this is actually working?

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language